# Построение рекомендательной системы content-based model на основе популярности 

![image](2023-10-01_13-50-10.png)

Допустим мы получили задание построить рекомендательную систему на основе контента, где необходимо:

* Для каждого продукта создать характеризующие его признаки.
* Найти показатель близости между всеми продуктами.
* Порекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил.

![Alt text](image.png)

Реализуем подобную рекомендательную систему на практике. Будем работать с [датасетом](https://github.com/Lidiya-cutie/Mini_project_model_Netflix/blob/master/netflix_titles.zip), содержащим информацию об оценивании фильмов на платформе Netflix.

Признаки в данных:

* *show_id* — id фильма,
* *type* — его тип (фильм или сериал),
* *title* — название,
* *director* — режиссер,
* *cast* — актерский состав,
* *country* — страна,
* *date_added* — дата добавления,
* *release_year* — год выхода на экраны,
* *rating* — рейтинг,
* *duration* — продолжительность,
* *listened_in* — жанр(-ы),
* *description* — описание.

Импортируем необходимые библиотеки:

In [1]:
import pandas as pd 
import numpy as np

Считываем датасет:

In [2]:
netflix_df = pd.read_csv('netflix_titles.zip')

В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход **TF-IDF** (*Term Frequency-Inverse Document Frequency*).

**Показатель TD-IDF** — это индикатор того, насколько релевантно слово в контексте документа.

Его можно определить следующим образом:

$$TF-IDF(слова) = TF(слова) * IDF(слова), \ где:$$

$\bullet \ TF \ слова = {{Количество \ раз, \ когда \ слово \ встретилось \ в \ тексте}\over{Количество \ всех \ слов \ в \ тексте}};$

$\bullet \ IDF \ слова = log({{Общее \ кол-во \ документов}\over{Кол-во \ документов, \ в \ которых \ встречается \ слово}}).$

Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

Таким образом:

$\bullet$ **Коэффициент будет выше**, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.

$\bullet$ **Коэффициент будет ниже**, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.

Если интересно подробнее изучить алгоритм создания такого представления, можно прочитать эту [статью](https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3).

Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая **функция из библиотеки sklearn** — импортируем её:

```python
from sklearn.feature_extraction.text import TfidfVectorizer

Далее учтём **стоп-слова**, т.е. предлоги и другие служебные части речи, которые не несут содержательной информации, и с учётом этого определим нашу модель:

```python
model = TfidfVectorizer(stop_words='english')

Заполним пропуски пустыми строками:

```python
df['description'] = df['description'].fillna('')

Трансформируем наши описания в матрицу:

```python
feature_matrix = model.fit_transform(df['description'])

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(stop_words='english')

netflix_df['description'] = netflix_df['description'].fillna('')

feature_matrix = model.fit_transform(netflix_df['description'])
print(f'Число столбцов в получившейся матрице {feature_matrix.shape[1]}.')

Число столбцов в получившейся матрице 17905


Теперь вычислим косинусную близость. Можно сделать это так:

In [5]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

Мы используем здесь *linear_kernel()*, а не [cosine_similarity()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity), так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, а TF-IDF создаёт уже нормализованные векторы.

Вернём индексацию и уберём дубликаты из данных:

In [6]:
indices = pd.Series(netflix_df.index,index=netflix_df['title']).drop_duplicates()

Теперь пропишем функцию для создания рекомендаций:

In [8]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return netflix_df['title'].iloc[ind_movie]

Например, если мы хотим найти рекомендации по фильму "Star Trek":

In [9]:
get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

Теперь найдём вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:

In [13]:
get_recommendations('Balto').iloc[1]

'Vroomiz'

Выводы:

* мы можем использовать фильтрацию по контенту, чтобы создавать рекомендации из аналогичных элементов;
* у нас получилось рекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил;
* мы не сможем порекомендовать продукты другого жанра или контента, если он сам не отметит подобные фильмы как понравившиеся.